## Annotator Metrics Evaluation
This notebook is intended to run metric evaluations to compare the ground truth annotations for annotators on the Janelia CellMap team. Specifically, annotators have labeled organelles in 3D EM datasets and we want to compare their evaluations to see if there are any outliers that may need further practice.

Data is expected to be subdivided into `groups` based on which organelles are labelled within them. `groups` are further divided into `crops` which correspond to specific region in the datasets that were labeled. `crops` contain the individual annotation images for each annotator in the region of interest. Optionally, the `crops` may also contain machine learning network predictions, refined segmentations from the predictions, and/or segmentations from Ariadne.

For a given `group` and `crop`, the code in this notebook can be used to perform an all-to-all comparison across all annotators and a variety of metrics. In particular, there are four main functions used to run the code:
1. `copy_data`: This function takes all the data from a source directory where it is organized in a predefined way and copies over into a new directory organized in a way better suited for the remainder of the analysis code. This only needs to be run once per `group`.
2. `calculate_all_to_all`: This function uses Dask to actually do the all-to-all comparison across the desired metrics, returns the results as well as plots the results. This is calculated for each `crop` within a `group`, or for specific crops if provided.
3. `create_variance_images`: For a given `group` and `crop`, this converts each annotation image into an n5 and calculates the variance across all manual annotations on a per-voxel basis, which itself is stored as an n5. This is calculated for each `crop` within a `group`, or for specific crops if provided.
4. `get_neuroglancer_view_of_crop`: Given a path to a `crop` n5, this will provide a neuroglancer link to actually view and compare the annotations, raw data and variance image to better enable annotation comparisons.

In [1]:
# Import the desired functions
from annotator_metrics.src.preprocessing import copy_data
from annotator_metrics.src.metrics_evaluations import calculate_all_to_all
from annotator_metrics.util.image_io import (
    create_variance_images,
    get_neuroglancer_view,
)

# Specify the group and crop of interest
group = "group2"
crop = "02"

# Specify base output path for where the data will be saved
output_base_path = "/groups/cellmap/cellmap/ackermand/annotation_and_analytics/"

# Copy the data
print(f"Copying {group}.")
copy_data(group=group, output_path=f"{output_base_path}/data/")

# Do the all-to-all calculations
print(f"Calculating all-to-all for {group} and crop {crop}:")
all_to_all_info, scores_info = calculate_all_to_all(
    group=group,
    input_path=f"{output_base_path}/data/",
    crop=crop,
    output_path=f"{output_base_path}/results/",
    num_workers=10,
)

# Create the variance image
print(f"Creating variance image for {group} and crop {crop}:")
create_variance_images(
    input_path=f"{output_base_path}/data/",
    group=group,
    output_path=f"{output_base_path}/results/n5s",
    crop=crop,
)

# Get a neuroglancer view of the crop. The http-served directory is /groups/cellmap/cellmap
print(f"Generating neuroglancer image for {group} and crop {crop}:")
get_neuroglancer_view(
    n5s_path=f"{output_base_path}/results/n5s",
    group = group
)


Copying group2:
Calculating all-to-all for group2 and 02:


Creating variance image for group2:


Generating neuroglancer image for group2 and 02:
